In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
ratings = pd.read_csv("digikala-ratings.csv")
ratings.columns = ['userId', 'rating','productId']
num_users = ratings['userId'].nunique()
#num_users = 1048389
ratings

,userId,rating,productId
0,53672599,1.0,252058
1,9897229,0.0,252058
2,38074516,0.0,3331597
3,18628562,0.0,3331329
4,53301258,3.0,3255700
...,...,...,...
1048570,14062382,0.0,64441
1048571,12178218,4.0,64441
1048572,7784354,3.0,64441
1048573,49779263,4.0,820989


In [3]:
def clean_titles(title):
    if isinstance(title, str):
        return re.sub(r"-", " ", title)
    return title

In [4]:
products = pd.read_csv("digikala-products.csv",low_memory=False)
products

,id,title_fa,Rate,Rate_cnt,Category1,Category2,Brand,Price,Seller,Is_Fake,min_price_last_month,sub_category
0,7096438,آبسلانگ مدل s5 بسته 250 عددی,90,4,آبسلانگ,NaN,متفرقه,634800,سلامت ساز راد,False,0,beauty
1,2845119,آبسلانگ مدل M-1 بسته 400 عددی,84,217,آبسلانگ,NaN,متفرقه,818800,مهر افزون,False,0,beauty
2,6117745,آبسلانگ مدل m50 مجموعه 500 عددی,88,14,آبسلانگ,NaN,متفرقه,920000,یانگوم,False,0,beauty
3,1912926,استند ابسلانگ مدل S01,78,25,آبسلانگ,NaN,متفرقه,1100000,بهراد طب بیدار,False,0,beauty
4,6335462,آبسلانگ نوری تسلامد مدل All-in-One,84,25,آبسلانگ,NaN,تسلا مد,1530000,دیجی‌کالا,False,0,beauty
...,...,...,...,...,...,...,...,...,...,...,...,...
1283491,8745218,جعبه نگهدارنده دارو مدل چوبی,90,24,یادآور و نگهدارنده قرص و دارو,NaN,متفرقه,2500000,وین شاپ,False,0,beauty
1283492,8159085,محفظه نگهداری قرص و پودر میوتنت مدل T,96,4,یادآور و نگهدارنده قرص و دارو,NaN,متفرقه,2150000,گیشا اسپورت,False,0,beauty
1283493,8137567,محفظه نگهداری قرص و پودر اسپرتر مدل S,70,2,یادآور و نگهدارنده قرص و دارو,NaN,متفرقه,2100000,گیشا اسپورت,False,0,beauty
1283494,11618907,محفظه نگهداری قرص و پودر مدل pill crusher,0,0,یادآور و نگهدارنده قرص و دارو,NaN,متفرقه,2300000,استور شاپ,False,0,beauty


In [5]:
products.drop(['Rate','Price','Is_Fake','min_price_last_month'], axis=1, inplace=True)
products['Category2'] = products['Category2'].fillna(" ")
products['title'] = products['title_fa']+" "+products['Category1']+" "+products['Category2']+" "+products['Brand']+" "+products['Seller']+" "+products['sub_category']
products.drop(['title_fa','Category1','Category2','Brand','Seller','sub_category'], axis=1, inplace=True)
products.rename(columns={'id':'productId'},inplace =True)
products['cleaned_title'] = products['title'].apply(clean_titles)
products.dropna(subset=['cleaned_title'], inplace=True)
products

,productId,Rate_cnt,title,cleaned_title
0,7096438,4,آبسلانگ مدل s5 بسته 250 عددی آبسلانگ متفرقه ...,آبسلانگ مدل s5 بسته 250 عددی آبسلانگ متفرقه ...
1,2845119,217,آبسلانگ مدل M-1 بسته 400 عددی آبسلانگ متفرق...,آبسلانگ مدل M 1 بسته 400 عددی آبسلانگ متفرق...
2,6117745,14,آبسلانگ مدل m50 مجموعه 500 عددی آبسلانگ متفر...,آبسلانگ مدل m50 مجموعه 500 عددی آبسلانگ متفر...
3,1912926,25,استند ابسلانگ مدل S01 آبسلانگ متفرقه بهراد ط...,استند ابسلانگ مدل S01 آبسلانگ متفرقه بهراد ط...
4,6335462,25,آبسلانگ نوری تسلامد مدل All-in-One آبسلانگ ت...,آبسلانگ نوری تسلامد مدل All in One آبسلانگ ت...
...,...,...,...,...
1283491,8745218,24,جعبه نگهدارنده دارو مدل چوبی یادآور و نگهدارند...,جعبه نگهدارنده دارو مدل چوبی یادآور و نگهدارند...
1283492,8159085,4,محفظه نگهداری قرص و پودر میوتنت مدل T یادآور و...,محفظه نگهداری قرص و پودر میوتنت مدل T یادآور و...
1283493,8137567,2,محفظه نگهداری قرص و پودر اسپرتر مدل S یادآور و...,محفظه نگهداری قرص و پودر اسپرتر مدل S یادآور و...
1283494,11618907,0,محفظه نگهداری قرص و پودر مدل pill crusher یادآ...,محفظه نگهداری قرص و پودر مدل pill crusher یادآ...


In [6]:
merge = ratings.merge(products)
merge["user_rating_cnt"] = merge.groupby('userId')['rating'].transform('count')
merge["user_rating_ave"] = merge.groupby('userId')['rating'].transform('mean')
merge["product_rating_ave"] = merge.groupby('productId')['rating'].transform('mean')
merge

,userId,rating,productId,Rate_cnt,title,cleaned_title,user_rating_cnt,user_rating_ave,product_rating_ave
0,53672599,1.0,252058,212,کتاب صوتی امپراطوری اپ اثر چاد مورتا کتاب صوتی...,کتاب صوتی امپراطوری اپ اثر چاد مورتا کتاب صوتی...,1,1.0,3.139815
1,9897229,0.0,252058,212,کتاب صوتی امپراطوری اپ اثر چاد مورتا کتاب صوتی...,کتاب صوتی امپراطوری اپ اثر چاد مورتا کتاب صوتی...,1,0.0,3.139815
2,50067836,4.0,252058,212,کتاب صوتی امپراطوری اپ اثر چاد مورتا کتاب صوتی...,کتاب صوتی امپراطوری اپ اثر چاد مورتا کتاب صوتی...,1,4.0,3.139815
3,42026542,5.0,252058,212,کتاب صوتی امپراطوری اپ اثر چاد مورتا کتاب صوتی...,کتاب صوتی امپراطوری اپ اثر چاد مورتا کتاب صوتی...,1,5.0,3.139815
4,39401085,4.0,252058,212,کتاب صوتی امپراطوری اپ اثر چاد مورتا کتاب صوتی...,کتاب صوتی امپراطوری اپ اثر چاد مورتا کتاب صوتی...,1,4.0,3.139815
...,...,...,...,...,...,...,...,...,...
1657543,14062382,0.0,64441,20,کتاب فلسفه چیست اثر ژیل دلوز کتاب فلسفه و روان...,کتاب فلسفه چیست اثر ژیل دلوز کتاب فلسفه و روان...,1,0.0,2.750000
1657544,12178218,4.0,64441,20,کتاب فلسفه چیست اثر ژیل دلوز کتاب فلسفه و روان...,کتاب فلسفه چیست اثر ژیل دلوز کتاب فلسفه و روان...,1,4.0,2.750000
1657545,7784354,3.0,64441,20,کتاب فلسفه چیست اثر ژیل دلوز کتاب فلسفه و روان...,کتاب فلسفه چیست اثر ژیل دلوز کتاب فلسفه و روان...,1,3.0,2.750000
1657546,49779263,4.0,820989,2,کتاب کار عربی پایه یازدهم اثر حبیب اله درویش ک...,کتاب کار عربی پایه یازدهم اثر حبیب اله درویش ک...,1,4.0,3.500000


In [7]:
ratings_matrix = merge[(merge['user_rating_cnt'] >= 4) & (merge['Rate_cnt'] >= 4)]
ratings_matrix

,userId,rating,productId,Rate_cnt,title,cleaned_title,user_rating_cnt,user_rating_ave,product_rating_ave
22586,37357749,4.0,2084527,679,کرم مرطوب کننده و آبرسان آردن بیوتی مدل Ha حجم...,کرم مرطوب کننده و آبرسان آردن بیوتی مدل Ha حجم...,5,4.0,3.870968
22587,37357749,4.0,2084527,679,کرم مرطوب کننده و آبرسان آردن بیوتی مدل Ha حجم...,کرم مرطوب کننده و آبرسان آردن بیوتی مدل Ha حجم...,5,4.0,3.870968
22588,37357749,4.0,2084527,679,کرم مرطوب کننده و آبرسان آردن بیوتی مدل Ha حجم...,کرم مرطوب کننده و آبرسان آردن بیوتی مدل Ha حجم...,5,4.0,3.870968
22589,37357749,4.0,2084527,679,کرم مرطوب کننده و آبرسان آردن بیوتی مدل Ha حجم...,کرم مرطوب کننده و آبرسان آردن بیوتی مدل Ha حجم...,5,4.0,3.870968
22590,37357749,4.0,2084527,679,کرم مرطوب کننده و آبرسان آردن بیوتی مدل Ha حجم...,کرم مرطوب کننده و آبرسان آردن بیوتی مدل Ha حجم...,5,4.0,3.870968
...,...,...,...,...,...,...,...,...,...
1651289,54319347,5.0,1416104,18912,کرم آبرسان و مرطوب کننده سوپکس مدل HYDRA PLUS ...,کرم آبرسان و مرطوب کننده سوپکس مدل HYDRA PLUS ...,5,5.0,3.863636
1651290,54319347,5.0,1416104,18922,کرم آبرسان و مرطوب کننده سوپکس مدل HYDRA PLUS ...,کرم آبرسان و مرطوب کننده سوپکس مدل HYDRA PLUS ...,5,5.0,3.863636
1651291,54319347,5.0,1416104,18912,کرم آبرسان و مرطوب کننده سوپکس مدل HYDRA PLUS ...,کرم آبرسان و مرطوب کننده سوپکس مدل HYDRA PLUS ...,5,5.0,3.863636
1651292,54319347,5.0,1416104,18912,کرم آبرسان و مرطوب کننده سوپکس مدل HYDRA PLUS ...,کرم آبرسان و مرطوب کننده سوپکس مدل HYDRA PLUS ...,5,5.0,3.863636


In [8]:
user_item_matrix = ratings_matrix.pivot_table(index = 'userId' , columns = 'productId', values = 'rating').fillna(0)
user_item_matrix

productId,37385,41480,45302,50113,74357,78013,81205,81207,81215,81218,...,11544320,11587507,11701176,11778118,11973587,12128936,12128979,12188803,12303066,12387198
userId,,,,,,,,,,,,,,,,,,,,,
1066727,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1800665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1840909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1979138,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2148643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54680060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54681427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54701725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


SVD Prediction Evaluation

In [9]:
n_factors =20
svd = TruncatedSVD(n_components=n_factors, random_state=42)
U = svd.fit_transform(user_item_matrix)
Sigma = svd.singular_values_
Vt = svd.components_

cumulative_sum = np.cumsum(Sigma)
total_sum = np.sum(Sigma)
limit = total_sum * 0.9
n_components_to_keep = np.argmax(cumulative_sum >= limit) + 1

sigma = Sigma[:n_components_to_keep]
u  = U[:, :n_components_to_keep]
vt = Vt[:n_components_to_keep, :]

user_item_matrix_pred = u @ np.diag(sigma) @ vt

predicted_ratings_df = pd.DataFrame(user_item_matrix_pred, 
                                    index=user_item_matrix.index, 
                                    columns=user_item_matrix.columns)
predicted_ratings_df

productId,37385,41480,45302,50113,74357,78013,81205,81207,81215,81218,...,11544320,11587507,11701176,11778118,11973587,12128936,12128979,12188803,12303066,12387198
userId,,,,,,,,,,,,,,,,,,,,,
1066727,-3.927569e-09,5.180796e-11,4.834710e-08,7.550442e-15,-1.700002e-08,-1.309031e-07,-8.764158e-10,-4.435585e-08,4.427680e-12,-8.111637e-06,...,6.408704e-15,-1.577778e-10,1.339711e-17,-3.702693e-10,8.826247e-17,-1.565043e-17,1.175424e-12,-1.723753e-13,3.848157e-13,2.183105e-12
1800665,3.450232e-06,-7.912532e-08,-3.274947e-06,-1.392825e-10,3.221624e-05,2.803782e-05,3.676368e-07,1.146821e-05,-4.826407e-10,5.351228e-04,...,1.747960e-12,-3.548874e-07,-2.547840e-15,-1.637570e-07,8.919943e-14,1.096542e-15,2.232410e-09,2.538539e-11,-1.146629e-09,-8.372474e-10
1840909,3.353312e-09,-4.039943e-10,-2.468242e-07,-5.125217e-13,9.038734e-08,-2.125809e-07,7.541153e-10,-1.176519e-08,1.186040e-11,-1.208164e-05,...,1.042718e-14,-7.918880e-10,-9.542414e-20,-1.352494e-09,1.497991e-16,-4.418696e-17,8.797931e-12,-1.510709e-13,-6.691235e-12,-1.681572e-12
1979138,-5.599803e-17,-4.591384e-18,7.111301e-15,-8.656926e-21,2.235197e-15,-1.293670e-14,-1.376194e-16,-3.504532e-15,6.333466e-19,-1.430693e-13,...,-4.959844e-22,1.930507e-17,1.919903e-24,-2.538071e-17,3.678987e-23,1.308373e-24,2.604356e-19,-7.757555e-21,-6.005866e-21,2.264491e-19
2148643,4.960778e-05,8.789370e-07,1.865152e-03,-1.143436e-09,1.681424e-04,1.214216e-03,5.806767e-06,-5.760061e-05,7.754593e-09,-5.240967e-03,...,-1.334235e-11,-1.692853e-06,5.321182e-14,-9.184351e-08,2.269299e-12,2.579224e-14,2.339834e-09,2.592631e-10,-5.942001e-09,-2.759432e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54680060,8.078569e-06,-5.640790e-08,1.713979e-04,-2.598868e-10,4.694395e-05,1.855430e-04,1.861695e-06,2.545744e-05,-5.311900e-09,-6.253663e-03,...,1.186739e-11,-7.608874e-07,-1.570330e-14,-3.039828e-07,-8.898207e-15,-2.439291e-14,2.371628e-09,4.218796e-11,-2.678576e-09,-2.497680e-09
54681427,-8.745664e-10,2.017822e-11,-5.614665e-09,3.797828e-14,-8.898000e-09,-4.346650e-09,-4.278574e-11,-1.041868e-10,-2.694706e-13,-4.608557e-07,...,1.438123e-16,1.112879e-10,-1.930474e-18,4.297181e-11,-7.117492e-17,1.810642e-18,-4.204545e-13,-4.947179e-15,2.418994e-13,6.040350e-14
54701725,-2.024888e-19,-2.460584e-20,-6.805440e-18,-6.575941e-24,4.601225e-18,-1.506808e-17,-1.601823e-19,-7.263052e-19,3.201009e-22,1.071215e-15,...,-1.281218e-24,8.330422e-21,1.557906e-27,-7.709608e-21,4.188264e-26,5.687951e-27,5.052250e-22,7.733841e-24,4.602831e-23,1.645286e-22


In [10]:
y_actual = user_item_matrix.to_numpy().flatten()
y_pred = user_item_matrix_pred.flatten()
rmse = np.sqrt(mean_squared_error(y_actual, y_pred))
rmse  
#1 => 0.4533773840189887
#2 => 0.6311597646056337
#3 => 0.7648572705246017
#4 => 0.8693262983140649
#5 => 0.9647436561388766
#10 => 1.26049580122614
#20 => 1.6831521558067513

1.6831521558067513

SVD Recommender

In [11]:
def svd_recommender(user_id,rate_matrix):
    rated_items = ratings_matrix[(ratings_matrix['userId'] == user_id) & (ratings_matrix['rating']>0)]['productId']
    user_predictions = predicted_ratings_df.loc[user_id]
    user_predictions = user_predictions[~user_predictions.index.isin(rated_items)]
    sorted_predictions = user_predictions.sort_values(ascending=False)
    recommendations_ids = sorted_predictions.head(10).index
    recommendations = ratings_matrix[ratings_matrix['productId'].isin(recommendations_ids)].drop_duplicates('productId')
    
    return recommendations
    

In [12]:
svd_recommendations = svd_recommender(54101641,ratings_matrix)
svd_recommendations

,userId,rating,productId,Rate_cnt,title,cleaned_title,user_rating_cnt,user_rating_ave,product_rating_ave
36341,52023415,4.0,3216751,347,شامپو بدن سینره مدل magnolia bloom حجم 250 میل...,شامپو بدن سینره مدل magnolia bloom حجم 250 میل...,4,4.0,3.812013
97204,48037581,5.0,2062289,2018,کرم آبرسان سوپکس مدل COCONUT حجم 250 میلی لیتر...,کرم آبرسان سوپکس مدل COCONUT حجم 250 میلی لیتر...,5,5.0,4.147727
160894,54156701,5.0,126148,15070,رول ضد تعریق زنانه شون مدل Pink Princess حجم 5...,رول ضد تعریق زنانه شون مدل Pink Princess حجم 5...,4,5.0,4.271186
220552,52866611,4.0,334664,3748,کاندوم کاپوت مدل 7Hot Time بسته 12 عددی بهداشت...,کاندوم کاپوت مدل 7Hot Time بسته 12 عددی بهداشت...,4,4.0,4.131429
244648,54346567,5.0,1104386,847,کاندوم سیکس مدل Super Ribbed بسته 12 عددی بهدا...,کاندوم سیکس مدل Super Ribbed بسته 12 عددی بهدا...,4,5.0,4.007000
363625,37566072,4.0,4472530,362,تونر پاک کننده صورت سرمد مدل اسطوخودوس حجم 140...,تونر پاک کننده صورت سرمد مدل اسطوخودوس حجم 140...,5,4.0,3.781750
717835,54564491,5.0,3439189,255,کیف نوار بهداشتی طرح دختر کد A029 بهداشت زنان ...,کیف نوار بهداشتی طرح دختر کد A029 بهداشت زنان ...,4,5.0,4.179487
1127664,54583980,5.0,588039,1593,کاندوم تاخیری یک ساعته کاپوت مدل Cool Time بست...,کاندوم تاخیری یک ساعته کاپوت مدل Cool Time بست...,4,5.0,3.989848
1222471,54245857,5.0,8470695,1545,کفش راحتی کد 210 کفش زنانه و مردانه کفش ورزشی ...,کفش راحتی کد 210 کفش زنانه و مردانه کفش ورزشی ...,5,5.0,3.976744
1228301,54576572,5.0,3941974,1842,کرم آبرسان سوپکس مدل Aloevera حجم 130 میلی لیت...,کرم آبرسان سوپکس مدل Aloevera حجم 130 میلی لیت...,4,5.0,4.138889


Content-based filtering

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
TFIDF = vectorizer.fit_transform(products['cleaned_title'])
TFIDF

<1283273x1934295 sparse matrix of type '<class 'numpy.float64'>'
	with 39255097 stored elements in Compressed Sparse Row format>

In [14]:
def landing_page_content_based_recommender(user_id):
    rated_items = ratings_matrix[(ratings_matrix['userId'] == user_id) & (ratings_matrix['rating']>0)]['productId']
    all_similar_items = []
    for product_id in rated_items:
        product_category = merge[merge['productId'] == product_id]['cleaned_title'].iloc[0]
        product_category_vec = vectorizer.transform([product_category])
        similarity = cosine_similarity(product_category_vec, TFIDF).flatten()
        indices = np.argpartition(similarity, -5)[-5:]
        similar_items = merge.iloc[indices]
        similar_items = similar_items[similar_items['productId'] != product_id]
        all_similar_items.append(similar_items)
    all_similar_items_df = pd.concat(all_similar_items).drop_duplicates('productId')
    recommendations = all_similar_items_df.sort_values(by='product_rating_ave', ascending=False)
    return recommendations

In [15]:
landing_page_recommendations = landing_page_content_based_recommender(37357749)
landing_page_recommendations

,userId,rating,productId,Rate_cnt,title,cleaned_title,user_rating_cnt,user_rating_ave,product_rating_ave
990306,50581204,5.0,471546,2085,فوم شستشوی صورت ژنوبایوتیک مدل Purifying حجم 1...,فوم شستشوی صورت ژنوبایوتیک مدل Purifying حجم 1...,3,5.0,4.387500
993746,51187708,5.0,7856404,35,پالت سایه چشم آنی لیدی مدل پری دریایی کد 2022 ...,پالت سایه چشم آنی لیدی مدل پری دریایی کد 2022 ...,1,5.0,4.309091
981443,34504444,4.0,6133600,57,کاسه رنگ ابرو مدل Pantene مجموعه 6 عددی برس‌ها...,کاسه رنگ ابرو مدل Pantene مجموعه 6 عددی برس‌ها...,1,4.0,4.240000
989728,51076361,4.0,4732353,744,کرم آبرسان مورینگا امو مدل Rejuvenating حجم 50...,کرم آبرسان مورینگا امو مدل Rejuvenating حجم 50...,3,4.0,4.135593
994354,12405106,5.0,38973,419,کتاب حواستو جمع کن 1، تطابق و هماهنگی اثر فهیم...,کتاب حواستو جمع کن 1، تطابق و هماهنگی اثر فهیم...,1,5.0,2.955333


In [16]:
def persian_content_based_recommender(product_id):
    product_category = products[products['productId']==product_id]['cleaned_title'].iloc[0]
    product_category_vec = vectorizer.transform([product_category])
    similarity = cosine_similarity(product_category_vec,TFIDF).flatten()
    indices = np.argpartition(similarity,-5)[-5:]
    similar_items = products.iloc[indices]
    similar_items = similar_items[similar_items['productId'] != product_id]
    recommendations = similar_items.sort_values(by='Rate_cnt', ascending=False)
    recommendations = recommendations.drop_duplicates('productId')
    return recommendations

In [17]:
recs = persian_content_based_recommender(64441)
recs

,productId,Rate_cnt,title,cleaned_title
1197666,64604,18,کتاب نیچه و فلسفه اثر ژیل دلوز نشر نی کتاب فلس...,کتاب نیچه و فلسفه اثر ژیل دلوز نشر نی کتاب فلس...
1192629,113952,7,کتاب فلسفه نقادی کانت اثر ژیل دلوز کتاب فلسفه ...,کتاب فلسفه نقادی کانت اثر ژیل دلوز کتاب فلسفه ...
1199600,443794,6,کتاب یک زندگی اثر ژیل دلوز نشر چشمه کتاب فلسفه...,کتاب یک زندگی اثر ژیل دلوز نشر چشمه کتاب فلسفه...
1184256,9085344,5,کتاب جهان اسپینوزا اثر ژیل دلوز نشر نی کتاب عل...,کتاب جهان اسپینوزا اثر ژیل دلوز نشر نی کتاب عل...


Hybrid Recommender For Landing Page

In [18]:
def hybrid_recommender(user_id):
    if user_id in ratings_matrix['userId'].values:
        svd_rec = svd_recommender(user_id,ratings_matrix)
        svd_rec = svd_rec.head(5)
        content_rec = landing_page_content_based_recommender(user_id)
        content_rec = content_rec.head(5)
        recommendations  = pd.concat([svd_rec,content_rec], axis=0).drop_duplicates('productId')
        return recommendations
    else:
        print("Not Enough Data!")
    
    

In [20]:
hybrid_recs= hybrid_recommender(37357749)
hybrid_recs

,userId,rating,productId,Rate_cnt,title,cleaned_title,user_rating_cnt,user_rating_ave,product_rating_ave
346054,51710181,5.0,5735394,226,وازلین فدک تانیش مدل BL2 حجم 100 میلی لیتر مجم...,وازلین فدک تانیش مدل BL2 حجم 100 میلی لیتر مجم...,5,5.0,4.125683
363625,37566072,4.0,4472530,362,تونر پاک کننده صورت سرمد مدل اسطوخودوس حجم 140...,تونر پاک کننده صورت سرمد مدل اسطوخودوس حجم 140...,5,4.0,3.781750
629358,54504485,5.0,1679460,8793,کاندوم کاپوت مدل BIG DOTS بسته 10 عددی بهداشت ...,کاندوم کاپوت مدل BIG DOTS بسته 10 عددی بهداشت ...,4,5.0,4.150000
641178,54564510,5.0,2092337,1344,کاور سری یدک مسواک برقی مدل آرال-فر بسته 4 عدد...,کاور سری یدک مسواک برقی مدل آرال فر بسته 4 عدد...,4,5.0,4.300699
717835,54564491,5.0,3439189,255,کیف نوار بهداشتی طرح دختر کد A029 بهداشت زنان ...,کیف نوار بهداشتی طرح دختر کد A029 بهداشت زنان ...,4,5.0,4.179487
990306,50581204,5.0,471546,2085,فوم شستشوی صورت ژنوبایوتیک مدل Purifying حجم 1...,فوم شستشوی صورت ژنوبایوتیک مدل Purifying حجم 1...,3,5.0,4.387500
993746,51187708,5.0,7856404,35,پالت سایه چشم آنی لیدی مدل پری دریایی کد 2022 ...,پالت سایه چشم آنی لیدی مدل پری دریایی کد 2022 ...,1,5.0,4.309091
981443,34504444,4.0,6133600,57,کاسه رنگ ابرو مدل Pantene مجموعه 6 عددی برس‌ها...,کاسه رنگ ابرو مدل Pantene مجموعه 6 عددی برس‌ها...,1,4.0,4.240000
989728,51076361,4.0,4732353,744,کرم آبرسان مورینگا امو مدل Rejuvenating حجم 50...,کرم آبرسان مورینگا امو مدل Rejuvenating حجم 50...,3,4.0,4.135593
994354,12405106,5.0,38973,419,کتاب حواستو جمع کن 1، تطابق و هماهنگی اثر فهیم...,کتاب حواستو جمع کن 1، تطابق و هماهنگی اثر فهیم...,1,5.0,2.955333
